In [30]:
from __future__ import print_function
__author__= 'Pablo Mandiola'

import os

import requests
try:
    import urllib2 as urlib
except ImportError:
    import urllib as urlib

import pandas as pd

In [2]:
puidata = os.getenv("PUIDATA")
if puidata is None:
    os.environ["PUIDATA"] = "{}/PUIdata".format(os.getenv("HOME"))
    puidata = os.getenv("PUIDATA")
    print("Warning: PUIDATA environmental variable not found and set by code, please review!")
print("PUIDATA: {}".format(puidata))

PUIDATA: /nfshome/pmb434/PUIdata


### LODES Data Structure

**File naming**

\[ST\]\_od_\[PART\]\_\[TYPE\]_\[YEAR\].csv.gz

\[ST\] = lowercase, 2-letter postal code for a chosen state

\[PART\] = Part of the state file, can have a value of either “main” or “aux”. Complimentary parts of
the state file, the main part includes jobs with both workplace and residence in the state
and the aux part includes jobs with the workplace in the state and the residence outside of
the state.

\[TYPE\] = Job Type, can have a value of “JT00” for All Jobs, “JT01” for Primary Jobs, “JT02” for
All Private Jobs, “JT03” for Private Primary Jobs, “JT04” for All Federal Jobs, or “JT05”
for Federal Primary Jobs.

\[YEAR\] = Year of job data. Can have the value of 2002-2015 for most states.

**Variable dictionary**

![](img/lodes_variable_dict.PNG)

### Download, Aggregate and Merge NYC LODES data

In [186]:
#005 - Bronx.
#047 - Kings (Brooklyn)
#061 - New York (Manhattan)
#081 - Queens.
#085 - Richmond (Staten Island)
nyc_boroughs = [5, 47, 61, 81, 85]

state = 'ny'
parts = ['main','aux']
jobs = 'JT00'
years = range(2010,2016)

censusblock_csv = 'nyc_jobs_by_censusblock.csv'

if not os.path.isfile(puidata + '/' + final_csv):
    print('Fetching all LODES files')
    
    nyc_lodes = pd.DataFrame(columns=['w_geocode'])
    
    for y in years:

        for p in parts:

            filename = '{}_od_{}_{}_{}.csv'.format(state,p,jobs,y)
            zipname = '{}.gz'.format(filename)
            url = 'https://lehd.ces.census.gov/data/lodes/LODES7/ny/od/{}'.format(zipname)

            if not os.path.isfile(puidata + '/' + zipname):
                print('Downloading {}'.format(zipname))
                os.system("curl -O " + url)
                os.system("mv " + zipname + " " + puidata)

            if not os.path.isfile(puidata + '/' + filename):
                # Unzip and move to puidata
                print('Unzipping {}'.format(zipname))
                os.system("gunzip -k " + puidata + "/" + zipname)

            # Check:
            if not os.path.isfile(puidata + "/" + filename):
                print ("WARNING!!! Something is wrong: {} is not there!".format(filename))
                break
            else:
                print ("{} in place, continue".format(filename))

            #Read file
            print('Reading {}'.format(filename))
            lodes = pd.read_csv(puidata + '/' + filename)

            #Generate county codes
            print('Getting County codes {}'.format(filename))
            lodes['County'] = (lodes.w_geocode - 360000000000000)//10000000000

            #Get rid of non-NYC Census Blocks
            print('Cleaning {}'.format(filename))
            lodes = lodes.loc[lodes.County.apply(lambda x: x in nyc_boroughs)]

            #Group by 
            print('Grouping {}'.format(filename))
            lodes = lodes.groupby(['w_geocode'])[['S000']].sum().reset_index().rename(columns={'S000':'{}_{}'.format(y, p)})

            #Merge
            print('Merging {}'.format(filename))
            nyc_lodes = nyc_lodes.merge(lodes, how='outer')
            nyc_lodes.fillna(0, inplace=True)

        #Total for the year
        print('Calculating total for {}'.format(y))
        nyc_lodes['{}_total'.format(y)] = nyc_lodes['{}_{}'.format(y, parts[0])] + nyc_lodes['{}_{}'.format(y, parts[1])]


    #Getting census_tracts for aggregating later
    nyc_lodes['w_census_tract_geoid'] = nyc_lodes.w_geocode // 10000

    #Saving file
    print('Saving file')
    nyc_lodes.to_csv(puidata + '/' + censusblock_csv, index=False)

    print('Done')
    
else:
    print('Already generated')
    nyc_lodes = pd.read_csv(puidata + '/' + censusblock_csv)

nyc_lodes.head()

ny_od_main_JT00_2010.csv in place, continue
Reading ny_od_main_JT00_2010.csv
Getting County codes ny_od_main_JT00_2010.csv
Cleaning ny_od_main_JT00_2010.csv
Grouping ny_od_main_JT00_2010.csv
Merging ny_od_main_JT00_2010.csv
ny_od_aux_JT00_2010.csv in place, continue
Reading ny_od_aux_JT00_2010.csv
Getting County codes ny_od_aux_JT00_2010.csv
Cleaning ny_od_aux_JT00_2010.csv
Grouping ny_od_aux_JT00_2010.csv
Merging ny_od_aux_JT00_2010.csv
Calculating total for 2010
ny_od_main_JT00_2011.csv in place, continue
Reading ny_od_main_JT00_2011.csv
Getting County codes ny_od_main_JT00_2011.csv
Cleaning ny_od_main_JT00_2011.csv
Grouping ny_od_main_JT00_2011.csv
Merging ny_od_main_JT00_2011.csv
ny_od_aux_JT00_2011.csv in place, continue
Reading ny_od_aux_JT00_2011.csv
Getting County codes ny_od_aux_JT00_2011.csv
Cleaning ny_od_aux_JT00_2011.csv
Grouping ny_od_aux_JT00_2011.csv
Merging ny_od_aux_JT00_2011.csv
Calculating total for 2011
ny_od_main_JT00_2012.csv in place, continue
Reading ny_od_main

,w_geocode,2010_main,2010_aux,2010_total,2011_main,2011_aux,2011_total,2012_main,2012_aux,2012_total,2013_main,2013_aux,2013_total,2014_main,2014_aux,2014_total,2015_main,2015_aux,2015_total,w_census_tract_geoid
0,360050002002004,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36005000200
1,360050002002010,9.0,0.0,9.0,14.0,0.0,14.0,3.0,0.0,3.0,2.0,0.0,2.0,3.0,0.0,3.0,4.0,0.0,4.0,36005000200
2,360050002003000,2.0,0.0,2.0,2.0,0.0,2.0,8.0,0.0,8.0,14.0,0.0,14.0,5.0,0.0,5.0,8.0,0.0,8.0,36005000200
3,360050002003006,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36005000200
4,360050002003008,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36005000200


### Aggregate to Zip Codes

Using HUD's Zip Codes crosswalk files

https://www.huduser.gov/portal/datasets/usps_crosswalk.html#data

In [187]:
filename = 'TRACT_ZIP_032015.xlsx'
url = 'https://www.huduser.gov/portal/datasets/usps/{}'.format(filename)

if not os.path.isfile(puidata + '/' + filename):
    print('Downloading {}'.format(filename))
    os.system("curl -O " + url)
    os.system("mv " + filename + " " + puidata)
    
# Check:
if not os.path.isfile(puidata + "/" + filename):
    print ("WARNING!!! Something is wrong: {} is not there!".format(filename))
else:
    print ("{} in place, continue".format(filename))

TRACT_ZIP_032015.xlsx in place, continue


In [188]:
tract_zip = pd.read_excel(puidata + "/" + filename)
tract_zip.head()

,TRACT,ZIP,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,1001020100,36067,1.000000,1.000000,1.000000,1.000000
1,1001020200,36008,0.011581,0.004190,0.026316,0.010003
2,1001020200,36067,0.409354,0.423184,0.921053,0.419140
3,1001020200,36068,0.579065,0.572626,0.052632,0.570857
4,1001020300,36067,1.000000,1.000000,1.000000,1.000000


In [189]:
data_columns = list(nyc_lodes.columns[1:-1])
nyc_tract_jobs = nyc_lodes.groupby('w_census_tract_geoid')[data_columns].sum().reset_index()

In [190]:
nyc_zipcode_jobs = nyc_tract_jobs.merge(tract_zip, left_on='w_census_tract_geoid', right_on='TRACT')
nyc_zipcode_jobs.head()

,w_census_tract_geoid,2010_main,2010_aux,2010_total,2011_main,2011_aux,2011_total,2012_main,2012_aux,2012_total,...,2014_total,2015_main,2015_aux,2015_total,TRACT,ZIP,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,36005000200,17.0,0.0,17.0,31.0,3.0,34.0,55.0,1.0,56.0,...,51.0,58.0,5.0,63.0,36005000200,10473,1.0,1.000000,1.0,1.000000
1,36005000400,171.0,10.0,181.0,221.0,5.0,226.0,243.0,14.0,257.0,...,310.0,447.0,20.0,467.0,36005000400,10473,1.0,1.000000,1.0,1.000000
2,36005001600,833.0,39.0,872.0,557.0,33.0,590.0,558.0,43.0,601.0,...,578.0,1327.0,67.0,1394.0,36005001600,10473,1.0,1.000000,1.0,1.000000
3,36005001900,4409.0,471.0,4880.0,4506.0,487.0,4993.0,5259.0,491.0,5750.0,...,5666.0,4908.0,643.0,5551.0,36005001900,10455,0.0,0.011236,0.0,0.002665
4,36005001900,4409.0,471.0,4880.0,4506.0,487.0,4993.0,5259.0,491.0,5750.0,...,5666.0,4908.0,643.0,5551.0,36005001900,10454,1.0,0.988764,1.0,0.997335


Multiply by BUS_RATIO: Percentage of business addresses of the Census Tract that are located inside the ZIP CODE

In [191]:
nyc_zipcode_jobs[data_columns] = nyc_zipcode_jobs[data_columns].multiply(nyc_zipcode_jobs.BUS_RATIO, axis=0)

In [192]:
nyc_zipcode_jobs = nyc_zipcode_jobs.groupby('ZIP')[data_columns].sum().reset_index()
nyc_zipcode_jobs.head()

,ZIP,2010_main,2010_aux,2010_total,2011_main,2011_aux,2011_total,2012_main,2012_aux,2012_total,2013_main,2013_aux,2013_total,2014_main,2014_aux,2014_total,2015_main,2015_aux,2015_total
0,10001,110844.034347,19581.224930,130425.259277,119934.398116,23090.888002,143025.286117,125107.527841,25479.839613,150587.367454,130289.410633,25957.475047,156246.885680,141243.473574,27495.652957,168739.126530,139210.585312,25517.325040,164727.910353
1,10002,17501.189156,1315.914654,18817.103810,18241.376674,1509.372059,19750.748733,18575.186672,1630.603800,20205.790472,17311.940049,1721.605455,19033.545504,19097.528421,1921.498079,21019.026500,21439.070402,1863.979643,23303.050046
2,10003,61530.840315,9363.819239,70894.659554,64739.789084,10308.860475,75048.649559,59786.418308,10273.453187,70059.871496,62081.330388,10712.407074,72793.737462,64501.109568,11508.768508,76009.878076,69158.840084,11036.073600,80194.913684
3,10004,29653.820952,7383.809509,37037.630460,31956.568737,7831.348067,39787.916803,34086.517676,8482.183181,42568.700857,32776.487794,8014.649468,40791.137261,38354.697164,8961.738961,47316.436125,40094.595260,8584.264357,48678.859617
4,10005,44953.776763,12602.323590,57556.100352,43606.040890,12769.476561,56375.517451,43311.874140,12738.632010,56050.506150,39880.206253,12228.742810,52108.949063,39087.680497,11178.336790,50266.017287,39821.496535,9739.560536,49561.057071


### Some of the jobs don't get allocated to any Zip Code

I *think* this is because the BUS_RATIO doesn't exactly sum up to 1 for each Census Tract

In [193]:
nyc_lodes.sum()[1:-1] - nyc_zipcode_jobs.sum()[1:]

2010_main      355.0
2010_aux        20.0
2010_total     375.0
2011_main      643.0
2011_aux        73.0
2011_total     716.0
2012_main      931.0
2012_aux       102.0
2012_total    1033.0
2013_main      806.0
2013_aux        71.0
2013_total     877.0
2014_main      941.0
2014_aux       105.0
2014_total    1046.0
2015_main     2238.0
2015_aux       159.0
2015_total    2397.0
dtype: float64

But the error is proportionally very small

In [194]:
(nyc_lodes.sum()[1:-1] - nyc_zipcode_jobs.sum()[1:]).sum()/nyc_lodes.sum().sum()

1.2605424421255257e-15

### Save final CSV

In [195]:
final_csv = 'nyc_jobs_by_zipcode.csv'

In [196]:
nyc_zipcode_jobs.to_csv(puidata + '/' + final_csv, index=False)